•	Data and environment structure 
o	Environment setting
o	Data loading (register the data)
o	Data modeling (register the data)
o	Analysis (register the model)
o	Data returning (register the data)
o	Creating requirements files 
•	Docker and docker-compose files
o	Create the docker-compose file 
o	Create the docker file
o	Build and run the dockerfile
•	Kubernetes and scheduling the process


In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
import pandas as pd
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# Number of samples
n = 1000

# Generate Retention data: binary values 0 or 1
retention = np.random.choice([0, 1], n)

# Generate Age data: integers between 18 and 65
age = np.random.randint(18, 66, n)

# Generate Female data: binary values 0 or 1
female = np.random.choice([0, 1], n)

# Generate Urban data: binary values 0 or 1
urban = np.random.choice([0, 1], n)

# Generate Income data: floating-point numbers skewed towards lower end
income = np.random.exponential(scale=20000, size=n)
income = np.clip(income, 0, 100000)

# Create a pandas DataFrame
df = pd.DataFrame(
    {
        "retention": retention,
        "age": age,
        "female": female,
        "urban": urban,
        "income": income,
    }
)

x = df[["age", "female", "urban", "income"]]
y = df["retention"]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

In [5]:
clf = XGBClassifier(objective="binary:logistic", n_estimators=100, seed=42)
clf.fit(x_train, y_train.ravel())

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/data.py:192: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[15:48:48] WARNING: /mnt/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [6]:
y_pred = clf.predict(x_test)

In [7]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Retrieve variables
subscription_id = os.getenv("subscription_id")
resource_group = os.getenv("resource_group")
workspace_name = os.getenv("workspace_name")
tenant_id = os.getenv("tenant_id")
client_id = os.getenv("client_id")
client_secret = os.getenv("client_secret")

In [8]:
from azureml.core import Workspace, Dataset
from azureml.core.authentication import ServicePrincipalAuthentication


svc_pr = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=client_id,
    service_principal_password=client_secret
)

# Access the workspace
workspace = Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    workspace_name=workspace_name,
    auth=svc_pr
)

datastore = workspace.get_default_datastore()

UserErrorException: UserErrorException:
	Message: You are currently logged-in to a93eafc8-c064-4a88-bc01-47e93068b42b tenant. You don't have access to d3ade6e0-a7be-4323-ace4-02b7b4da2451 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 []. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to a93eafc8-c064-4a88-bc01-47e93068b42b tenant. You don't have access to d3ade6e0-a7be-4323-ace4-02b7b4da2451 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = \n []. \n Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk."
    }
}

In [10]:
ds = Dataset.Tabular.register_pandas_dataframe(
    dataframe=df,
    name="data_scheduler",
    description="Initial data",
    target=datastore,
)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/3672a148-1798-4a55-80f8-8dc5e671de07/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
